<a href="https://colab.research.google.com/github/Seongjin1225/AI_SCHOOL_9/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D/11%EC%9B%94%2002%EC%9D%BC/11%EC%9B%94_2%EC%9D%BC_%EB%B3%B5%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 라이브러리 탑재
import folium
import pandas as pd

# 기본 서울 지도 그래프 보기
seoul = folium.Map(location=[37.55, 126.98], zoom_start = 12)
seoul

# 승하차 인원 파일 읽어오기
data = pd.read_csv('/content/drive/MyDrive/AI스쿨 파일/seoul-metro-2021.logs.csv')
data.head()

# 역 기준으로 합치기
station_sum = data.groupby('station_code').sum(numeric_only=False)  # numeric_only=False 조건 있어서 숫자형 아닌 것도 포함
station_sum

# 지하철역 파일 읽어오기
station_info = pd.read_csv('/content/drive/MyDrive/AI스쿨 파일/seoul-metro-station-info.csv')
station_info.head()

# 필요한 컬럼만 추출하기(역코드, 위도, 경도)
station_info = station_info[['station.code','geo.latitude','geo.longitude']]
station_info.head()

# 데이터를 승하차 인원 데이터와 맞추기 위해 색인 설정(set_index)
station_info = station_info.set_index('station.code')
station_info.head()

# 승하차 인원 데이터와 지하철 역 데이터 합치기(join)
joined_data = station_sum.join(station_info)
joined_data

,timestamp,people_in,people_out,geo.latitude,geo.longitude
station_code,,,,,
150,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,12697273,12109991,37.554648,126.972559
151,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,5997344,6030491,37.564718,126.977108
152,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,9638952,9284693,37.570161,126.982923
153,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,7704599,7090896,37.571607,126.991806
154,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,6651283,6609055,37.570926,127.001849
...,...,...,...,...,...
2824,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,3206211,3097722,37.445210,127.156866
2825,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,1395919,1495603,37.440918,127.147564
2826,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,1614590,1475010,37.437428,127.140722


In [ ]:
# 승하차 인원 지도에 히트맵으로 나타내기
from folium.plugins import HeatMap

HeatMap(data=joined_data[['geo.latitude','geo.longitude','people_in']]).add_to(seoul)
seoul

In [ ]:
seoul_out = folium.Map(location=[37.55, 126.98], zoom_start = 12)

HeatMap(data=joined_data[['geo.latitude','geo.longitude','people_out']]).add_to(seoul_out)
seoul_out

In [ ]:
# 위 2개의 지도는 결과가 같아 자세히 알기 어려움
# 출퇴근 시간(9시 이전, 17시 이후)로 범위 설정 후 확인

# 출근 시간 데이터
morning_data = data[pd.to_datetime(data.timestamp).dt.hour < 9]
morning_data

# 퇴근 시간 데이터
evening_data = data[pd.to_datetime(data.timestamp).dt.hour > 17]
evening_data

,timestamp,station_code,people_in,people_out
13,2021-01-01T18:00:00.000+09:00,150,658,437
14,2021-01-01T19:00:00.000+09:00,150,579,425
15,2021-01-01T20:00:00.000+09:00,150,479,354
16,2021-01-01T21:00:00.000+09:00,150,510,307
17,2021-01-01T22:00:00.000+09:00,150,445,134
...,...,...,...,...
1941548,2021-12-31T19:00:00.000+09:00,2828,74,263
1941549,2021-12-31T20:00:00.000+09:00,2828,73,145
1941550,2021-12-31T21:00:00.000+09:00,2828,95,209
1941551,2021-12-31T22:00:00.000+09:00,2828,54,138


In [ ]:
# 역 기준 그룹화하여 데이터 합치기
# 그룹화
morning_group = morning_data.groupby('station_code').sum(numeric_only=False)
evening_group = evening_data.groupby('station_code').sum(numeric_only=False)

# 합치기
morning_join = morning_group.join(station_info)
evening_join = evening_group.join(station_info)

In [ ]:
# 시각화(출근시간 승차 인원 시각화)
morning_seoul_in = folium.Map(location=[37.55, 126.98], zoom_start = 12)
HeatMap(data=morning_join[['geo.latitude','geo.longitude','people_in']]).add_to(morning_seoul_in)
morning_seoul_in


In [ ]:
# 시각화(출근시간 하차 인원 시각화)
morning_seoul_out = folium.Map(location=[37.55, 126.98], zoom_start = 12)
HeatMap(data = morning_join[['geo.latitude', 'geo.longitude', 'people_out']]).add_to(morning_seoul_out)
morning_seoul_out

In [ ]:
# 시각화(퇴근시간 승차 인원 시각화)
evening_seoul_in = folium.Map(location=[37.55, 126.98], zoom_start = 12)
HeatMap(data=evening_join[['geo.latitude', 'geo.longitude', 'people_in']]).add_to(evening_seoul_in)
evening_seoul_in

In [ ]:
# 시각화(퇴근시간 하차 인원 시각화)
evening_seoul_out = folium.Map(location=[37.55, 126.98], zoom_start = 12)
HeatMap(data=evening_join[['geo.latitude', 'geo.longitude', 'people_out']]).add_to(evening_seoul_out)
evening_seoul_out

In [ ]:
# 점심시간의 지하철 승하차 현황을 분석해봅니다. - 11시부터 2시 사이
lunch_data = data[(pd.to_datetime(data.timestamp).dt.hour <= 14) & (pd.to_datetime(data.timestamp).dt.hour >=11)]
lunch = lunch_data.groupby('station_code').sum(numeric_only=False)
lunch_join = lunch.join(station_info)

# 승차 인원 확인
lunch_seoul_in = folium.Map(location=[37.55, 126.98], zoom_start = 12)
HeatMap(data = lunch_join[['geo.latitude', 'geo.longitude', 'people_in']]).add_to(lunch_seoul_in)
lunch_seoul_in

# 하차 인원 확인
lunch_seoul_out = folium.Map(location=[37.55, 126.98], zoom_start = 12)
HeatMap(data = lunch_join[['geo.latitude', 'geo.longitude', 'people_out']]).add_to(lunch_seoul_out)
lunch_seoul_out



AttributeError: ignored

In [ ]:
# 문제 2
# 문제 1에서 가공한 데이터 가지고 히트맵 대신 원하는 역에 동그라미 ㅇㅇ
# 조건
# --> 점심시간 하차인원 250만 이상인 역
#     동그라미 테두리는 붉은색, 내부는 붉은색 음영
#     반지름은 하차인원의 0.01%
lunch_circle = folium.Map(location=[37.55, 126.98], zoom_start=12)

for idx, row in lunch_join[['geo.latitude', 'geo.longitude', 'people_out']].iterrows():
    if int(row['people_out']) >= 2_500_000:
        folium.Cirlce(location=[str(row['geo.latitude']), str(row['geo.langitude'])], color = 'red', fill=True,
                      radius=(int(row['people_out']))/10000).add_to(lunch_circle)
lunch_circle

In [ ]:
# 4~6시 프로젝트
# 지도 만들기
friday = folium.Map(location=   , zoom_start = 12)
saturday = folium.Map(location=[37.55, 126.98], zoom_start=10)

# 금요일 19시 ~  정보 뽑아내기
friday_night = data[(pd.to_datetime(data.timestamp).dt.weekday == 4) & (pd.to_datetime(data.timestamp).dt.hour >= 19)]
# friday_night

# 토요일 5시 ~ 8시 추출
saturday_morning = data[(pd.to_datetime(data.timestamp).dt.weekday == 5) & ((pd.to_datetime(data.timestamp).dt.hour >= 5) & (pd.to_datetime(data.timestamp).dt.hour <=8 ))]
#saturday_morning

# 역기준 그룹화
fri_group = friday_night.groupby('station_code').sum(numeric_only=False)
sat_group = saturday_morning.groupby('station_code').sum(numeric_only=False)

# 데이터 합치기
fri_join = fri_group.join(station_info)
fri_join
# sat_join = sat_group.join(station_info)
# sat_join

AttributeError: ignored

In [ ]:
# 금요일 저녁 하차 히트맵 표시
HeatMap(data=fri_join[['geo.latitude','geo.longitude','people_out']]).add_to(friday)
friday

In [ ]:
# 토요일 아침 승차 히트맵 표시
HeatMap(data=fri_join[['geo.latitude','geo.longitude','people_in']]).add_to(saturday)
saturday